In [ ]:
import pandas as pd
import numpy as np

# para la división de datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

# codificación de características categóricas
from sklearn.preprocessing import LabelEncoder

# para escalado de características
from sklearn.preprocessing import StandardScaler

# para entrenamiento del modelo de Random Forest
from sklearn.ensemble import RandomForestRegressor

# para validación cruzada del modelo
from sklearn.model_selection import cross_val_score

# para evaluación del modelo
from sklearn.metrics import r2_score

# para visualización de datos
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

# importas más modelos, para evaluaciones
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# lectura del df
df = pd.read_csv("./data.csv")
df.head(2)

df_modelo = df.copy()
df_modelo.drop(columns=['Nombre'], inplace=True)
print(df_modelo.isna().sum())
print(df_modelo.duplicated().sum())

# elimino la columna que voy a predecir
# separo los datos de prueba y los a predecir
X = df_modelo.drop('Valor Actual', axis=1)
y = df_modelo['Valor Actual']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# proceso de encoding
encoders = {}
cat_features = ['Equipo', 'Posicion']

for feature in cat_features:
    encoders[feature] = LabelEncoder()
    X_train[feature + "_codificadas"] = encoders[feature].fit_transform(X_train[feature])
    X_test[feature + "_codificadas"] = encoders[feature].transform(X_test[feature])

# eliminación de columnas categóricas para mejor funcionamiento del modelo
X_train.drop(columns=['Equipo', 'Posicion'], inplace=True)
X_test.drop(columns=['Equipo', 'Posicion'], inplace=True)

# selecciono las variables que tienen mayor correlación 
corr_matrix = pd.concat([X_train, y_train], axis=1).corr()
high_corr_features = corr_matrix.index[abs(corr_matrix['Valor Actual']) > 0.5]
X_train = X_train[high_corr_features.drop('Valor Actual')]
X_test = X_test[high_corr_features.drop('Valor Actual')]

# Estandarizo los datos de los conjuntos de entrenamiento 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# creo y entreno el modelo
rf_reg = RandomForestRegressor(n_estimators=150, random_state=42)
rf_reg.fit(X_train_scaled, y_train)

# se controla el rendimiento del modelo y se analiza 
cv_scores = cross_val_score(rf_reg, X_train_scaled, y_train, cv=5, scoring='r2')
print(f"Puntuaciones de validación cruzada R^2: {cv_scores}")
print(f"Puntuación media de validación cruzada R^2: {np.mean(cv_scores)}")

# se realizan las predicciones
y_pred = rf_reg.predict(X_test_scaled)
r2_score_test = r2_score(y_test, y_pred)
print(f"Puntuación R^2 en conjunto de prueba: {r2_score_test}")

# visualización de las predicciones 
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test, color='red', label='Valores Reales')
plt.scatter(y_test, y_pred, color='blue', label='Predicciones')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='gray', linestyle='--')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Predicciones vs Valores Reales')
plt.legend()
plt.grid(True)
plt.show()

# Leer el DataFrame
df = pd.read_csv('./data.csv')

# Crear una pequeña constante para evitar la división por cero
small_constant = 1e-5

# Creacion de variables sintéticas centradas en el valor del jugador
df['Goles_por_Aparicion'] = df['Goles'] / (df['Apariciones'] + small_constant)
df['Asistencias_por_Aparicion'] = df['Asistencias'] / (df['Apariciones'] + small_constant)
df['Proporcion_Valor'] = df['Valor Actual'] / df['Valor Maximo']

# Llenar valores nulos resultantes de divisiones por cero
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# Mostrar las primeras filas del DataFrame para verificar las nuevas variables
df.head(2)

# Separar las características y la variable objetivo
X = df.drop(['Valor Actual', 'Nombre', 'Equipo', 'Posicion'], axis=1)
y = df['Valor Actual']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definir los modelos a probar
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor()
}

# Evaluar los modelos utilizando validación cruzada
results = {}
for model_name, model in models.items():
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    results[model_name] = scores.mean()

# Mostrar los resultados
results

# Realizar PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train_scaled)

# Mostrar la varianza explicada por cada componente
explained_variance = pca.explained_variance_ratio_
print("Varianza:", explained_variance)

# Mostrar las cargas de los primeros dos componentes
pca_components = pd.DataFrame(pca.components_, columns=X.columns, index=['PC1', 'PC2'])
print("Cargas de los primeros dos componentes:")
pca_components

# Crear y entrenar el modelo utilizando validación cruzada
model = RandomForestRegressor(random_state=42)
cv_scores = cross_val_score(model, X_train, y_train, cv=10)

# Mostrar los resultados de la validación cruzada
print('Puntuaciones de validación cruzada:', cv_scores)
print('Media de puntuaciones:', np.mean(cv_scores))

# Entrenamiento del modelo sin validación cruzada
model.fit(X_train, y_train)
score_sin_cv = model.score(X_test, y_test)

# Resultados
print('Score sin validación cruzada:', score_sin_cv)
print('Score con validación cruzada (media):', np.mean(cv_scores))
